# Overview

This notebook provides an example of the new features provided in release v.0.1.3.

For a better interactive viewing experience, you could open this notebook with google colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sina-mansour/connectome-based-smoothing/blob/main/notebooks/example-v.0.1.3.ipynb)

The codes for high-resolution connectome mapping are adopted from [this repository](https://github.com/sina-mansour/neural-identity). If you are using the high-resolution connectome mapping tools, please make sure to cite the original article too:

**Mansour, L. Sina, et al. "High-resolution connectomic fingerprints: mapping neural identity and behavior." *NeuroImage* (2021): 117695.**


## Importing packages

Cells in this section import the required python packages:

Package installations (for google colab only)

In [ ]:
# install package on google colab
! pip install -U Connectome-Spatial-Smoothing


Package imports

In [1]:
# First, import basic python packages
import os
import sys
import numpy as np

# Additionally, import the CSS package
from Connectome_Spatial_Smoothing import CSS as css


Data download (for google colab only)

In [ ]:
# load data on google colab
! rm -r connectome-spatial-smoothing
! git clone --depth 1 https://github.com/sina-mansour/connectome-spatial-smoothing.git
os.chdir('/content/connectome-spatial-smoothing/notebooks/')


## Loading files

Cells in this section present the neuroimaging data and explain how to load the required files.

In [2]:
# Get paths for all files:
main_dir = os.path.join(os.path.abspath(os.path.dirname(os.getcwd())), 'code/Connectome_Spatial_Smoothing')

tractography_file = os.path.abspath('{}/data/sample/sample_tractography_10k.tck'.format(main_dir))
tractography_file_mni = os.path.abspath('{}/data/sample/sample_tractography_10k_MNI.tck'.format(main_dir))

left_native_surface_file = os.path.abspath('{}/data/sample/sample.native.L.white.32k_fs_LR.surf.gii'.format(main_dir))
right_native_surface_file = os.path.abspath('{}/data/sample/sample.native.R.white.32k_fs_LR.surf.gii'.format(main_dir))

left_MNI_surface_file = os.path.abspath('{}/data/sample/sample.MNI152.L.white.32k_fs_LR.surf.gii'.format(main_dir))
right_MNI_surface_file = os.path.abspath('{}/data/sample/sample.MNI152.R.white.32k_fs_LR.surf.gii'.format(main_dir))

warp_file = os.path.abspath('{}/data/sample/sample_standard2acpc_dc.nii.gz'.format(main_dir))
cifti_file = os.path.abspath('{}/data/templates/cifti/ones.dscalar.nii'.format(main_dir))

brain_atlas_file = os.path.abspath('{}/data/templates/atlas/Glasser360.32k_fs_LR.dlabel.nii'.format(main_dir))


## Warping tractography

Cells in this section explain the new tractography warping feature:

We have now exposed some of the codes within the script that could be used to warp a `.tck` tractography using a nonlinear warp.

This function can be used to check whether the warping is being performed as expected:

In [3]:
css.save_warped_tractography(
    tractography_file,
    warp_file,
    tractography_file_mni,
)


The function above warped the input tractography file and stores the output accordingly. This way, we could check the original and warped files to ensure that warping is working as expected.

The picture below shows the original native space tractography (`sample_tractography_10k.tck`) overlaid on the native T1 image when visualized with Mrtrix's mrview:

<img src="../static/images/tractography_native_overlay.png" alt="Tractography" style="width: 500px;"/>


This next picture shows the warped MNI space tractography (`sample_tractography_10k_MNI.tck`) generated by the script above, overlaid on the MNI T1 image when visualized with Mrtrix's mrview:

<img src="../static/images/tractography_MNI_overlay.png" alt="Tractography" style="width: 500px;"/>

As evident in these two pictures the warp is working as expected. In other words, the streamlines after warping are still closely terminating at the white-matter gray-matter boundary which is a sign of appropriate nonlinear warping from the native space to the standard MNI space.


## Smoothing and connectome mapping with subcortex

Cells in this section show the newly added option to include subcortex and cerebellum as volumetric nodes (voxels):

New (backwards compatible) options are now added to map connectomes that include the subcortex and to also smooth those connectomes:

Alternatively, we could map the connectomes directly onto the MNI surface by providing an additional nonlinear warp:

In [4]:
# Map high-resolution connectome onto native surfaces:
high_resolution_connectome = css.map_high_resolution_structural_connectivity(
    tractography_file,
    left_MNI_surface_file,
    right_MNI_surface_file,
    warp_file=warp_file,
    subcortex=True
)


Fri May 27 02:03:29 2022: [INFO] loading tractography file.
Fri May 27 02:03:29 2022: [INFO] track file loaded: /home/sina/Documents/Research/Codes/CBS/connectome-based-smoothing/code/Connectome_Spatial_Smoothing/data/sample/sample_tractography_10k.tck
Fri May 27 02:03:30 2022: [INFO] endpoints extracted: #10000
Fri May 27 02:03:30 2022: [INFO] endpoints warped: #10000
Fri May 27 02:03:32 2022: [INFO] closest brainordinates located
Fri May 27 02:03:32 2022: [INFO] outliers located: #3097 outliers (30.97%, with threshold 2mm)
Fri May 27 02:03:32 2022: [INFO] creating sparse incidence matrix
Fri May 27 02:03:32 2022: [INFO] sparse matrix generated


We can check that the dimension of this new matrix is increased as a result of inclusion of volumetric nodes:

In [5]:
# Check the output matrix
high_resolution_connectome


<91282x91282 sparse matrix of type '<class 'numpy.float32'>'
	with 13783 stored elements in Compressed Sparse Row format>

Similarly, the smoothing kernel can now be created with the subcortex included:

In [6]:
%%time
smoothing_kernel = css.compute_smoothing_kernel(left_MNI_surface_file, right_MNI_surface_file, fwhm=3, epsilon=0.01, subcortex=True)


CPU times: user 7.76 s, sys: 822 ms, total: 8.59 s
Wall time: 1min 42s


In [7]:
help(css.compute_smoothing_kernel)

Help on function compute_smoothing_kernel in module Connectome_Spatial_Smoothing.CSS:

compute_smoothing_kernel(left_surface_file, right_surface_file, fwhm, epsilon=0.01, cifti_file='/home/sina/.local/lib/python3.8/site-packages/Connectome_Spatial_Smoothing/data/templates/cifti/ones.dscalar.nii', subcortex=False, volume_smoothing='integrated')
    Compute the CSS smoothing kernel on the cortical surfaces.
    
    Args:
    
        left_surface_file: The left hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        right_surface_file: The right hemisphere's surface in fs-LR 32k space (surf.gii format)
    
        fwhm: The full width at half maximum (FWHM) in mm.
    
        epsilon: The kernel truncation threshold.
    
        cifti_file: [optional, default: HCP-YA template] path to a cifti file, this template is used
                    to determine the high-resolution structure to exclude the medial wall and potentially
                    integrate subcortical vol

Finally, we could then use the computed smoothing kernel to perform CSS on a high-resolution connectome including the volumetric cifti structure:

$A_s = F_s A {F_s}^T$

In [8]:
smoothed_high_resolution_connectome = css.smooth_high_resolution_connectome(high_resolution_connectome, smoothing_kernel)
